## Import libraries

In [9]:
import json
import xmltodict
from dotenv import load_dotenv
from funcs import * # import the function from the package
import azure.cosmos.cosmos_client as cosmos_client
import time

## import variables

In [2]:
load_dotenv()
# get CosmosDB settings
HOST = f'https://{os.environ.get("COSMOSDB_HOST")}.documents.azure.com:443/'
MASTER_KEY = os.environ.get("COSMOSDB_KEY") # Master Key for CosmosDB
DATABASE_ID = os.environ.get("COSMOSDB_DATABASE_ID") # CosmosDB database
CONTAINER_ID_METABOLITES = os.environ.get("COSMOSDB_METABOLITES_CONTAINER_ID") # CosmosDB container for file records
CONTAINER_ID_METADATA = os.environ.get("COSMOSDB_METADATA_CONTAINER_ID") # CosmosDB container for metadata records
CONTAINER_ID_USERS = os.environ.get("COSMOSDB_USERS_CONTAINER_ID") # CosmosDB container for user records
cosmosdb_client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="INFO408", user_agent_overwrite=True)

## Create database

In [3]:
database, container_metabolites, container_users = create_database(cosmosdb_client, DATABASE_ID, CONTAINER_ID_METABOLITES, CONTAINER_ID_USERS)

Database with id 'INFO408Project' was found
Container with id 'Metabolites' was found
Container with id 'Users' was found
Admin user with email "noval419@student.otago.ac.nz" already exists

Database create function is complete



## Data preparation

### Convert xml file to json

Steps to follow:
1. Create "data" folder
2. Download serum metabolites zip file from: https://hmdb.ca/downloads
3. Unpack the zip file in "data" folder

In [17]:
# open serum metabolites xml file
with open("data\\serum_metabolites.xml", encoding='utf-8') as xml_file:
     
    data_dict = xmltodict.parse(xml_file.read()) # convert to dictionary
    json_data = json.dumps(data_dict) # convert to json
    
# Write the json data file
with open("data\data.json", "w") as json_file:
    json_file.write(json_data)

Make sure that data.json file has been created. Expected run time is around 1.5-2 minutes, and the file size is around 829Mb.

The file saving step is not strictly necessary as we could just continue to file structure conversion directly.

### Convert the schema

As per subject matter expert instructions not all fields are required in the new database. We need to load the file again and change document schema.

#### Load json data file

In [4]:
# Read the JSON data from the file  
with open("data\data.json", "r") as json_file:  
    json_data = json_file.read()  
  
# Convert the JSON data back into a Python dictionary  
data_dict = json.loads(json_data)

#### Print sample document

In [16]:
data_dict['hmdb']['metabolite'][0]

{'version': '5.0',
 'creation_date': '2005-11-16 15:48:42 UTC',
 'update_date': '2021-10-13 17:34:04 UTC',
 'accession': 'HMDB0000001',
 'status': 'quantified',
 'secondary_accessions': {'accession': ['HMDB00001',
   'HMDB0004935',
   'HMDB0006703',
   'HMDB0006704',
   'HMDB04935',
   'HMDB06703',
   'HMDB06704']},
 'name': '1-Methylhistidine',
 'description': '1-Methylhistidine, also known as 1-MHis or 1MH, belongs to the class of organic compounds known as histidine and derivatives. 1MH is also classified as a methylamino acid. Methylamino acids are primarily proteogenic amino acids (found in proteins) which have been methylated (in situ) on their side chains by various methyltransferase enzymes. Histidine can be methylated at either the N1 or N3 position of its imidazole ring, yielding the isomers 1-methylhistidine (1MH; also referred to as pi-methylhistidine) or 3-methylhistidine (3MH; tau-methylhistidine), respectively. There is considerable confusion with regard to the nomenclat

#### Convert to new schema

In [6]:
new_docs_list = [] # list of new documents to be inserted
# iterate through each document in the data_dict
for doc in data_dict['hmdb']['metabolite']:
    new_docs_list.append(create_new_document(doc)) # create new document and append to list
new_docs_list[0] # print first document

{'id': 'HMDB0000001',
 'common_name': '1-Methylhistidine',
 'description': '1-Methylhistidine, also known as 1-MHis or 1MH, belongs to the class of organic compounds known as histidine and derivatives. 1MH is also classified as a methylamino acid. Methylamino acids are primarily proteogenic amino acids (found in proteins) which have been methylated (in situ) on their side chains by various methyltransferase enzymes. Histidine can be methylated at either the N1 or N3 position of its imidazole ring, yielding the isomers 1-methylhistidine (1MH; also referred to as pi-methylhistidine) or 3-methylhistidine (3MH; tau-methylhistidine), respectively. There is considerable confusion with regard to the nomenclature of the methylated nitrogen atoms on the imidazole ring of histidine and other histidine-containing peptides such as anserine. In particular, older literature (mostly prior to the year 2000) designated anserine (Npi methylated) as beta-alanyl-N1-methyl-histidine, whereas according to s

#### Save new json file

In [7]:
with open("data\data_new.json", "w") as json_file:
        json_file.write(json.dumps(new_docs_list))

## Upload documents to the new database

### Load newly converted documents from json file

In [8]:
# Read the JSON data from the file  
with open("data\data_new.json", "r") as json_file:  
    json_data = json_file.read()  
  
# Convert the JSON data back into a Python dictionary  
data = json.loads(json_data) 


### Upload documents to the CosmosDB database

In [10]:
start = time.time()
for doc in data:
    upload_item(container_metabolites, doc)
print("Total time to upload: ", time.time() - start, " seconds")

Total time to upload:  2787.2774310112  seconds


Unfortunately, Python SDK for Azure CosmosDB NoSQL API does not support batch upload. Plus free account has limitations to the rate of the upload.

### Test read an item from the database

In [12]:
print(read_item(container_metabolites, 'HMDB0000001', 'HMDB0000001'))

{'id': 'HMDB0000001', 'common_name': '1-Methylhistidine', 'description': '1-Methylhistidine, also known as 1-MHis or 1MH, belongs to the class of organic compounds known as histidine and derivatives. 1MH is also classified as a methylamino acid. Methylamino acids are primarily proteogenic amino acids (found in proteins) which have been methylated (in situ) on their side chains by various methyltransferase enzymes. Histidine can be methylated at either the N1 or N3 position of its imidazole ring, yielding the isomers 1-methylhistidine (1MH; also referred to as pi-methylhistidine) or 3-methylhistidine (3MH; tau-methylhistidine), respectively. There is considerable confusion with regard to the nomenclature of the methylated nitrogen atoms on the imidazole ring of histidine and other histidine-containing peptides such as anserine. In particular, older literature (mostly prior to the year 2000) designated anserine (Npi methylated) as beta-alanyl-N1-methyl-histidine, whereas according to sta